In [78]:
MLFLOW_TRACKING_URI = "../models/mlruns"
MLFLOW_RUN_ID = "871e33a94e9a42819d93d28209cfca7e"

LOG_DATA_PKL = "data.pkl"
LOG_MODEL_PKL = "model.pkl"
LOG_METRIC_PKL = "metrics.pkl"


YEAR_DIC = {1990: 1, 1991: 2, 1992: 3, 1993: 4, 1994: 5, 
            1995: 6, 1996: 7, 1997: 8, 1998: 9, 1999: 10, 
            2000: 11, 2001: 12, 2002: 13, 2003: 14, 2004: 15,
            2005: 16, 2006: 17, 2007: 18, 2008: 19, 2009: 20,
            2010: 21, 2011: 22, 2012: 23, 2013: 24, 2014: 25, 
            2015: 26, 2016: 27, 2017: 28}

TRANSMISSION_DICT = {'manual': 1, 'automatic': 2,
                     'automated_manual': 3, 'direct_drive':4}

In [95]:
# Load packages
import pandas as pd
import numpy as np

import pickle
import os

import mlflow
from mlflow.tracking import MlflowClient

___
## Intilalize


In [4]:
# Intialize client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()

In [5]:
run = client.get_run(MLFLOW_RUN_ID)
artifacts_path = run.info.artifact_uri
artifacts_path

'../models/mlruns/0/871e33a94e9a42819d93d28209cfca7e/artifacts'

### Load artifacts

In [9]:
# Load model
model_path = os.path.join(artifacts_path, LOG_MODEL_PKL)
with open (model_path, "rb") as file:
    model = pickle.load(file)

In [10]:
model.keys()

dict_keys(['model_description', 'model_details', 'model_object'])

In [11]:
# Load Data pickle
data_path = os.path.join(artifacts_path, LOG_DATA_PKL)
with open(data_path, "rb") as file:
    data = pickle.load(file)

In [12]:
data.keys()

dict_keys(['data_path', 'training_indices', 'test_indices', 'feature_names', 'target_names'])

In [14]:
# Unpack variables
features_names = pd.Series(data["feature_names"])
regressor = model["model_object"]

___
## Sample Entry

In [108]:
# [make, year, engine_hp, engine_cylinders, transmission_type, highway_mpg, city_mpg]
sample = ["mercedes-benz", 2014, 201, 4, 'manual', 34, 20]

In [109]:
sample_df = pd.DataFrame(features_names, columns=['features'])
sample_df['value'] = sample_df['features'].isin(sample)
sample_df

,features,value
0,year,False
1,engine_hp,False
2,engine_cylinders,False
3,transmission_type,False
4,speed,False
5,acura,False
6,alfa_romeo,False
7,aston_martin,False
8,audi,False
9,bentley,False


In [110]:
# Changing the values in sample dataframe
sample_df.iloc[[0],[1]] = YEAR_DIC[sample[1]]
sample_df.iloc[[1],[1]] = sample[2]
sample_df.iloc[[2],[1]] = sample[3]
sample_df.iloc[[3],[1]] = TRANSMISSION_DICT[sample[4]]
sample_df.iloc[[4],[1]] = (sample[5] + sample[6])/2

In [111]:
sample_df.replace({False:0, True:1}, inplace=True)

In [112]:
sample_df.set_index("features", inplace=True)

In [113]:
sample_df = sample_df.transpose()#.values
sample_df

features,year,engine_hp,engine_cylinders,transmission_type,speed,acura,alfa_romeo,aston_martin,audi,bentley,...,rolls-royce,saab,scion,spyker,subaru,suzuki,tesla,toyota,volkswagen,volvo
value,25.0,201.0,4.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


___
## Predict

In [114]:
predictions = regressor.predict(sample_df.values)
predictions

array([10.62379613])

In [117]:
car_price = round(np.expm1(predictions)[0], 3)
print(f"The estimated price of your car= ({car_price}) USD")

The estimated price of your car= (41100.345) USD
